![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F05+-+TensorFlow&file=05Tools+-+Prediction+-+Batch.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05Tools%20-%20Prediction%20-%20Batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F05%2520-%2520TensorFlow%2F05Tools%2520-%2520Prediction%2520-%2520Batch.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05Tools%20-%20Prediction%20-%20Batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/05%20-%20TensorFlow/05Tools%20-%20Prediction%20-%20Batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 05Tools: Prediction - Batch
Predictions from models created in the 05 series of notebooks.

This notebook is part of collection of examples that showcase many ways to serve models:
- Online:
    - Vertex AI Endpoints: Python, REST, CLI (gcloud): [05Tools - Prediction - Online.ipynb](./05Tools%20-%20Prediction%20-%20Online.ipynb)
    - Local with TensorFlow ModelServer: [05Tools - Prediction - Local.ipynb](./05Tools%20-%20Prediction%20-%20Local.ipynb)
    - Custom: Build a custom container with TensorFlow ModelServer: [05Tools - Prediction - Custom.ipynb](./05Tools%20-%20Prediction%20-%20Custom.ipynb)
        - Remote Service with Cloud Run
        - Local Service with Docker Run
- Batch: (**THIS NOTEBOOK**) [05Tools - Prediction - Batch.ipynb](./05Tools%20-%20Prediction%20-%20Batch.ipynb)
    - BigQuery ML Model Import
    - Vertex AI Batch Prediction Jobs

**Prerequisites:**
-  At least 1 of the notebooks in this series [05, 05a-05i]

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05tools_pred_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05tools_pred_console.png" width="45%">
</p>

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
EXPERIMENT = '05_predictions'
SERIES = '05'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [3]:
from google.cloud import aiplatform
from google.cloud import bigquery
from datetime import datetime

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [5]:
BUCKET = PROJECT_ID
DIR = f"temp/{EXPERIMENT}"

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get a Model For Predictions
This project already has a model serving online predictions at a Vertex AI Endpoint.  This section will use the endpoint to retrieve the deployed model and get its information to use for batch prediction methods in this notebook.

### Get Endpoint

[Endpoint Properties and Methods](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint):

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [7]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
endpoint = endpoints[0]

In [8]:
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/1961322035766362112?project=statmike-mlops-349915


### Get Model at Endpoint
Using the model on the endpoint for the current series:

In [9]:
endpoint

resource name: projects/1026793852137/locations/us-central1/endpoints/1961322035766362112

In [10]:
#endpoint.list_models()[0]

In [11]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

### Review Model Information

In [12]:
model.display_name

'05_05h'

In [13]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_05_05h'

In [14]:
model.version_id

'1'

In [15]:
model.version_description

'run-20220927230247-6'

In [16]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05_05h@1'

In [17]:
model.supported_input_storage_formats

['jsonl', 'bigquery', 'csv', 'tf-record', 'tf-record-gzip', 'file-list']

In [18]:
model.name

'model_05_05h'

In [19]:
model.uri

'gs://statmike-mlops-349915/05/05h/models/20220927230247/6/model'

In [20]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}/versions/{model.version_id}/properties?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_05_05h/versions/1/properties?project=statmike-mlops-349915


#### Review Model Information Using the `aiplatform_v1` Model Client
It may also be helpful to try the [ModelServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.model_service.ModelServiceClient) in version 1 of the client to review the model attributes.  Here is example code for trying this.

Curious about client versions and layers?  Check out this tip document [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [22]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
ModelClientv1 = aiplatform_v1.ModelServiceClient(client_options = client_options)

ModelClientv1.get_model(
    name = model.versioned_resource_name
)

name: "projects/1026793852137/locations/us-central1/models/model_05_05h@1"
display_name: "05_05h"
predict_schemata {
}
metadata {
}
container_spec {
  image_uri: "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_deployment_resources_types: SHARED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1664323764
  nanos: 618427000
}
update_time {
  seconds: 1664323768
  nanos: 500624000
}
deployed_models {
  endpoint: "projects/1026793852137/locations/us-central1/endpoints/1961322035766362112"
  deployed_model_id: "6805735083375329280"
}
etag: "AMEw9yNfyB4gRyRK-Cg1ZIP8JArru29MZ7L-57j

---
## Vertex AI Batch Prediction Jobs

Create a [Vertex AI Batch Predictions](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#aiplatform_batch_predict_custom_trained-python) Job in one of these ways:
- Directly with [aiplatform.BatchPredictionJob.create()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.BatchPredictionJob)
- From a Model with [aiplatform.Model.batch_predict()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_batch_predict)
- From JobServiceClient with [aiplatform_v1.JobServiceClient.create_batch_prediction_job()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.job_service.JobServiceClient#google_cloud_aiplatform_v1_services_job_service_JobServiceClient_create_batch_prediction_job)
- From JobServiceClient with [aiplatform_v1beta1.JobServiceClient.create_batch_prediction_job()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.job_service.JobServiceClient#google_cloud_aiplatform_v1beta1_services_job_service_JobServiceClient_create_batch_prediction_job)

---
### Batch Prediction Job with BigQuery Tables

Using the Vertex AI SDK to request batch predictions with a BigQuery Source and Destination.  Records from the BigQuery source table will be formated as instances in the form ([documentation](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#bigquery)):
```
{"instances": [[<col 1 value>, <col 2 value>, ... ], ...]}
```

This is not the correct form for the model in this series which expects:
```
{"instances": [{col1_name: col1_value, col2_name: col2_value, ...}, ...]}
```

A later section in this notebook will show how to manually export the records from BigQuery as JSONL and run a batch prediction job using a GCS storage location and the import the results to BigQuery.  Another way to handle this custom situation is a container that first pre-processes the BigQuery records which can be simplified using a [Custom Prediction Routine](https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines).

For example purposes, the batch prediction job with BigQuery source and destination is run in the following cell and result are reviewed to show how errors are reported during batch predictions.

In [24]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
bqBatchJob = model.batch_predict(
    job_display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    #model_name = model.versioned_resource_name,
    labels = model.labels,
    instances_format = 'bigquery',
    predictions_format = 'bigquery',
    bigquery_source = f'bq://{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}',
    bigquery_destination_prefix = f'bq://{BQ_PROJECT}.{BQ_DATASET}',
    machine_type = 'n1-standard-8', #DEPLOY_COMPUTE,
    accelerator_count = 0,
    starting_replica_count = 10,
    max_replica_count = 10,
    batch_size = 500,
    sync = False #if True the call will wait for the job to complete
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/3862622556406153216
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/1026793852137/locations/us-central1/batchPredictionJobs/3862622556406153216')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/3862622556406153216?project=1026793852137
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/3862622556406153216 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/3862622556406153216 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/3862622556406153216 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredicti

In [26]:
bqBatchJob.output_info

bigquery_output_dataset: "bq://statmike-mlops-349915.fraud"
bigquery_output_table: "predictions_2022_10_14T04_12_16_032Z_180"

In [33]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{bqBatchJob.output_info.bigquery_output_table}_errors`
    LIMIT 10
"""
errors = bq.query(query = query).to_dataframe()
errors

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits,prediction_error
0,711,-0.431349,1.027694,2.670816,2.084787,-0.274567,0.286856,0.152110,0.200872,-0.596505,...,0.384610,-0.147132,-0.087100,0.101117,0.077944,0.0,0,a0ee62da-0f4a-47ca-8a0b-59f5e6abdf5c,TEST,Post request fails. Cannot get predictions. Er...
1,160340,-2.184687,3.150965,-1.632775,2.778267,0.163796,0.345102,-1.267995,-1.145605,-2.929760,...,0.546978,-0.296219,-0.399505,-0.924709,-0.256977,0.0,0,a4bfff59-ecb5-42b4-88f0-7128c27a790f,TEST,Post request fails. Cannot get predictions. Er...
2,39715,-1.592017,-0.541795,1.700835,-1.127247,1.549061,-0.940941,-0.173953,0.287038,-0.540361,...,-0.282122,0.153988,0.796829,-0.104723,0.051838,0.0,0,3a9775bc-9fb0-439d-8fc6-2384962a6f4d,TEST,Post request fails. Cannot get predictions. Er...
3,75176,1.235603,0.041383,0.675286,0.836279,-0.675016,-0.657342,-0.154209,-0.067491,0.602617,...,0.396205,0.324557,0.182930,-0.017115,0.014979,0.0,0,68367ea9-bf0a-4213-b1b2-9ac8b850b211,TEST,Post request fails. Cannot get predictions. Er...
4,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST,Post request fails. Cannot get predictions. Er...
5,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0,0,c8a5b93a-1598-4689-80be-4f9f5df0b8ce,TEST,Post request fails. Cannot get predictions. Er...
6,49561,1.333331,-0.845997,1.161578,-0.610965,-1.635783,-0.198304,-1.331531,0.212857,-0.208834,...,0.367347,0.398274,-0.059810,0.041674,0.011653,0.0,0,91c4e6ed-5a1a-4abf-916f-367d95cd074a,TEST,Post request fails. Cannot get predictions. Er...
7,61902,-0.902417,1.304038,2.199079,2.781943,-0.476967,1.368389,-0.585997,1.008109,-0.799174,...,-0.523843,0.147613,0.251030,0.238152,0.077121,0.0,0,ddcc473f-9608-4984-a40a-bc0889eed501,TEST,Post request fails. Cannot get predictions. Er...
8,118591,2.201278,0.125460,-2.904359,0.100054,1.347522,-0.746611,0.852297,-0.410838,-0.213915,...,-1.302901,0.811794,0.790992,-0.133376,-0.119079,0.0,0,55478ab8-7225-4d92-8bbd-fbb6754ff7a4,TEST,Post request fails. Cannot get predictions. Er...
9,143397,-1.167002,1.165700,3.192449,4.686220,-0.848641,2.413812,-1.303251,1.203409,-0.725670,...,-0.275264,0.248086,0.668773,0.202353,0.078456,0.0,0,6d827fa9-0ce5-4f91-bfb7-0a49d2a336c2,TEST,Post request fails. Cannot get predictions. Er...


In [34]:
errors['prediction_error'].iloc[0]

'Post request fails. Cannot get predictions. Error: Exceeded retries: Non-OK result 400 ({\n    "error": "instances is a plain list, but expecting list of objects as multiple input tensors required as per tensorinfo_map"\n}) from server, retry=3, ellapsed=0.02s.'

---
### Batch Prediction Job With JSONL
This process will export a BigQuery table to JSONL, use a Vertex AI Batch Prediction Job to create predictions, then import the result into BigQuery

#### BigQuery Extract To JSONL in GCS
The Batch Prediction job will validate the provided JSONL to the model and will not accept additional columns.  It is also important to note that when BigQuery exports to JSONL it wraps INT64 values in qoutes which makes them appear as strings.  For this reason, I first cast the INT64 columns to FLOAT64 before exporting.

In [90]:
# Exports to JSON appear to wrap INT64 types in qoutes
job = bq.query(
    query = f"""
        CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_json` AS
            SELECT * EXCEPT(Time, Class, transaction_id, splits), CAST(Time AS FLOAT64) AS Time
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    """
)
job.result()

In [22]:
ds = bigquery.DatasetReference(BQ_PROJECT, BQ_DATASET)
tb = ds.table(f'{BQ_TABLE}_json')
extractJob = bq.extract_table(
    source = tb,
    destination_uris = [f'gs://{BUCKET}/{SERIES}/{EXPERIMENT}/data/jsonl/{BQ_TABLE}.json'],
    job_config = bigquery.job.ExtractJobConfig(destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON)
)

In [23]:
extractJob.result()

ExtractJob<project=statmike-mlops-349915, location=us-central1, id=57a152de-15b2-47ce-bc13-cb15cb957348>

#### Batch Prediction Job

In [25]:
model.labels

{'series': '05',
 'experiment_name': 'experiment-05-05h-tf-classification-dnn',
 'run_name': 'run-20220927230247-6',
 'experiment': '05h'}

In [26]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
batchJob = aiplatform.BatchPredictionJob.create(
    job_display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    model_name = model.versioned_resource_name,
    labels = model.labels,
    instances_format = 'jsonl',
    predictions_format = 'jsonl',
    gcs_source = f'gs://{BUCKET}/{SERIES}/{EXPERIMENT}/data/jsonl/{BQ_TABLE}.json',
    gcs_destination_prefix = f'gs://{BUCKET}/{SERIES}/{EXPERIMENT}/data/jsonl/predictions_{TIMESTAMP}',
    machine_type = 'n1-standard-8', #DEPLOY_COMPUTE,
    accelerator_count = 0,
    starting_replica_count = 1,
    max_replica_count = 10,
    batch_size = 1000,
    sync = False #if True the call will wait for the job to complete
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/999720777072771072?project=1026793852137


In [27]:
batchJob.wait()

BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/999720777072771072 current state:
JobState.JOB

In [47]:
print(f'Review the Batch Prediction Job in the Console here:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/batch-predictions/{batchJob.name}?project={PROJECT_ID}')

Review the Batch Prediction Job in the Console here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/batch-predictions/999720777072771072?project=statmike-mlops-349915


#### Move the Prediction to a BigQuery Table: Predictions > BigQuery

In [28]:
batchJob.output_info.gcs_output_directory

'gs://statmike-mlops-349915/05/05_predictions/data/jsonl/predictions_20220928164702/prediction-05_05h-2022_09_28T09_47_02_215Z'

In [29]:
job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE, #.WRITE_APPEND, #.WRITE_TRUNCATE,
    create_disposition = bigquery.CreateDisposition.CREATE_IF_NEEDED,
    autodetect = True
)

In [30]:
tb = ds.table(f'{BQ_TABLE}_predictions_{TIMESTAMP}')
load_job = bq.load_table_from_uri(
    source_uris = f"{batchJob.output_info.gcs_output_directory}/*",
    destination = tb,
    job_config = job_config
)
load_job.result()

LoadJob<project=statmike-mlops-349915, location=us-central1, id=c520096f-d1c4-4eb7-9a67-79fddde41612>

In [31]:
job = bq.query(
    query = f"""
        CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_predictions_{TIMESTAMP}` AS
            SELECT instance.*, confidence, predicted_Class
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_predictions_{TIMESTAMP}`
            CROSS JOIN
            UNNEST(prediction) as confidence WITH OFFSET predicted_Class
            WHERE confidence > 0.5
    """
)
job.result()

In [32]:
bq.query(query = f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_predictions_{TIMESTAMP}` LIMIT 5").to_dataframe()

,Time,Amount,V28,V27,V25,V23,V22,V13,V20,V18,...,V14,V8,V19,V11,V6,V21,V4,V1,confidence,predicted_Class
0,33641,0.0,0.028651,0.048237,0.167130,0.096092,-0.120130,-1.692193,-0.305254,-0.439433,...,-0.045731,0.400944,-1.188879,-1.025257,0.580852,-0.078653,2.677320,1.081841,0.999834,0
1,142287,0.0,-0.056739,0.011784,-0.256187,0.150500,1.301533,-0.259683,-0.068230,-0.438197,...,-0.184497,0.030861,0.537967,0.961910,-0.602462,0.443190,-0.904821,2.133280,0.999989,0
2,67511,0.0,0.041391,0.300076,-0.368140,-0.163398,0.223127,-0.916773,-0.245192,-0.706260,...,0.800525,0.913627,-0.742845,-1.366311,-0.775504,0.073028,-0.375810,-1.585593,0.999999,0
3,146702,0.0,0.264933,0.353284,-1.047032,0.006383,1.513855,-1.727013,-0.335867,0.357580,...,2.076218,0.939516,0.076008,-0.403928,-1.684399,0.537311,-0.556876,-1.090239,1.000000,0
4,78582,0.0,0.036349,0.044793,0.307404,0.042671,-0.071119,0.322105,-0.145716,-0.235315,...,-0.470986,0.148990,-0.844085,-0.906068,0.157896,-0.083325,2.707886,1.126433,0.999672,0


---
## Batch Predictions: BigQuery ML

For TensorFlow models under 250MB using BigQuery with `ML.PREDICT` is possible after importing the model.  More on model import with BigQuery ML (BQML) can be found [here](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow).

Load a model to BigQuery and use BQML to create predictions.  Need the URI of TensorFlow model.

### Import Model Into BigQuery

In [35]:
query = f'''
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{model.name}`
    OPTIONS(
        MODEL_TYPE = 'TENSORFLOW',
        MODEL_PATH = '{model.uri}/*')
'''

In [36]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.fraud.model_05_05h`
    OPTIONS(
        MODEL_TYPE = 'TENSORFLOW',
        MODEL_PATH = 'gs://statmike-mlops-349915/05/05h/models/20220927230247/6/model/*')



In [37]:
job = bq.query(query = query)
job.result()
(job.ended-job.started).total_seconds()

5.129

### Generate Predictions With BigQuery (`ml.predict`)

In [38]:
query = f'''
SELECT *
FROM ML.PREDICT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.{model.name}`, (
        SELECT * 
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
        WHERE splits='TEST' AND Class = 1
        LIMIT 10
    )
)
'''
results = bq.query(query = query).to_dataframe()
results

,prediction_layer,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,"[0.00019910602713935077, 0.9998008012771606]",85285,-7.030308,3.421991,-9.525072,5.270891,-4.024630,-2.865682,-6.989195,3.791551,...,0.036943,-0.355519,0.353634,1.042458,1.359516,-0.272188,0.0,1,0a3b566f-e662-4cd0-b702-99299890cb0f,TEST
1,"[0.022684935480356216, 0.9773150682449341]",56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,e17f6ee4-8dd8-4a38-9f51-e1bbf1e1aa2a,TEST
2,"[0.0003789803013205528, 0.9996210932731628]",43369,-3.365319,2.426503,-3.752227,0.276017,-2.305870,-1.961578,-3.029283,-1.674462,...,-0.248502,0.126550,0.104166,-1.055997,-1.200165,-1.012066,88.0,1,125e7617-a79a-468f-af1b-f184544347f4,TEST
3,"[0.004454766400158405, 0.9955453276634216]",143354,1.118331,2.074439,-3.837518,5.448060,0.071816,-1.020509,-1.808574,0.521744,...,-0.021910,-0.376560,0.192817,0.114107,0.500996,0.259533,1.0,1,17e4d066-124e-4a22-84fb-40b8b3456adf,TEST
4,"[1.502191116742324e-05, 0.9999849200248718]",93888,-10.040631,6.139183,-12.972972,7.740555,-8.684705,-3.837429,-11.907702,5.833273,...,-0.567343,0.843012,0.549938,0.113892,-0.307375,0.061631,1.0,1,0a4e27c9-89e7-4588-86fd-70790cb3d45f,TEST
5,"[4.970725058228709e-06, 0.9999949336051941]",20332,-15.271362,8.326581,-22.338591,11.885313,-8.721334,-2.324307,-16.196419,0.512882,...,1.085617,-1.039797,-0.182006,0.649921,2.149247,-1.406811,1.0,1,5810b45a-602e-4113-b675-a0dcccaed0d5,TEST
6,"[0.0031228025909513235, 0.9968771934509277]",7551,0.316459,3.809076,-5.615159,6.047445,1.554026,-2.651353,-0.746579,0.055586,...,-0.583813,-0.219845,1.474753,0.491192,0.518868,0.402528,1.0,1,2e70bcc4-4a04-44c5-82c1-5ee6061196c5,TEST
7,"[3.1510986445937306e-05, 0.9999684691429138]",13126,-2.880042,5.225442,-11.063330,6.689951,-5.759924,-2.244031,-11.199975,4.014722,...,0.795255,-0.778379,-1.646815,0.487539,1.427713,0.583172,1.0,1,ab20db30-3b79-4ebb-ac94-2f98e1eabee9,TEST
8,"[2.3832196802686667e-06, 0.999997615814209]",152036,-4.320609,3.199939,-5.799736,6.502330,0.378479,-1.948246,-2.167860,-0.728207,...,-0.136940,-0.620072,0.642531,0.280717,-2.649107,0.533641,1.0,1,1803e0ce-b531-4919-a2d3-cf0d07172166,TEST
9,"[1.351359291135168e-08, 1.0]",100298,-22.341889,15.536133,-22.865228,7.043374,-14.183129,-0.463145,-28.215112,-14.607791,...,1.412928,0.382801,0.447154,-0.632816,-4.380154,-0.467863,1.0,1,7742da54-3f4c-4e8b-bccc-2d55ac0edd7e,TEST
